In [1]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)

In [2]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib

In [3]:
import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

ts_info_fullfn = debris_prms.ts_fp + debris_prms.roi + '_debris_tsinfo.nc'

if os.path.exists(debris_prms.ts_fp) == False:
    os.makedirs(debris_prms.ts_fp)
    
debug = False

print(ts_info_fullfn)

/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/../output/ts_tif/02_ts_data/02_debris_tsinfo.nc


In [4]:
rgiid_list = []
rgiid_fn_list = []
for i in os.listdir(debris_prms.mb_binned_fp):
    if i.endswith('mb_bins.csv'):
        region = int(i.split('.')[0])
        if region in debris_prms.roi_rgidict[debris_prms.roi]:    
            if region < 10:
                rgiid_list.append(i[0:7])
            else:
                rgiid_list.append(i[0:8])
            rgiid_fn_list.append(i)
        

# rgiid_list = ['18.00718']
# rgiid_fn_list = ['18.00718_mb_bins.csv']

rgiid_list = sorted(rgiid_list)
rgiid_fn_list = sorted(rgiid_fn_list)



main_glac_rgi = debris_prms.selectglaciersrgitable(rgiid_list)
main_glac_rgi['CenLon_360'] = main_glac_rgi['CenLon']
main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'])
main_glac_rgi['bin_fn'] = rgiid_fn_list

176 glaciers in region 2 are included in this model run: ['00280', '00737', '00914', '01104', '01152', '01158', '01161', '01290', '01291', '01297', '01339', '01397', '01441', '01654', '01665', '01685', '01727', '01811', '01812', '01922', '01923', '02107', '02348', '02360', '02386', '02432', '02526', '02527', '02533', '02550', '02551', '02616', '02636', '02686', '02745', '02747', '02752', '02784', '02857', '02894', '02897', '02947', '02948', '02966', '03099', '03102', '03157', '03520', '03578', '03581'] and more
This study is focusing on 176 glaciers in region [2]


In [5]:
# Merge with debris cover stats
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])
dc_shp.reset_index(inplace=True, drop=True)

dc_areaperc_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_%'].values))
main_glac_rgi['DC_Area_%'] = main_glac_rgi.RGIId.map(dc_areaperc_dict).fillna(0)
dc_area_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_v2'].values))
main_glac_rgi['DC_Area_v2'] = main_glac_rgi.RGIId.map(dc_area_dict).fillna(0)
main_glac_rgi

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float,CenLon_360,bin_fn,DC_Area_%,DC_Area_v2
GlacNo,,,,,,,,,,,,,,,,,,,,,
0,279,RGI60-02.00280,-122.57300,49.88620,2,2,5.938,1671,2340,1949,...,0,0,20059999,280,02.00280,2.00280,237.42700,2.00280_mb_bins.csv,7.90,304435
1,736,RGI60-02.00737,-116.52800,50.21440,2,3,5.536,2237,3104,2586,...,0,0,20059999,737,02.00737,2.00737,243.47200,2.00737_mb_bins.csv,8.14,288223
2,913,RGI60-02.00914,-123.85700,50.31570,2,2,4.372,1056,2233,1486,...,0,0,20059999,914,02.00914,2.00914,236.14300,2.00914_mb_bins.csv,19.99,784568
3,1103,RGI60-02.01104,-122.60500,50.41950,2,2,3.016,1850,2539,2054,...,0,0,20059999,1104,02.01104,2.01104,237.39500,2.01104_mb_bins.csv,8.92,169332
4,1151,RGI60-02.01152,-123.89000,50.34100,2,2,12.021,1280,2505,1908,...,0,0,20059999,1152,02.01152,2.01152,236.11000,2.01152_mb_bins.csv,9.40,680881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,18764,RGI60-02.18765,-121.01578,48.31119,2,4,4.845,1523,2642,2125,...,0,0,19589999,18765,02.18765,2.18765,238.98422,2.18765_mb_bins.csv,11.44,485905
172,18769,RGI60-02.18770,-121.00437,48.07234,2,4,2.125,1845,2490,2253,...,0,0,19849999,18770,02.18770,2.18770,238.99563,2.18770_mb_bins.csv,12.49,114329
173,18777,RGI60-02.18778,-121.05735,48.35698,2,4,2.924,1613,2196,1891,...,0,0,19589999,18778,02.18778,2.18778,238.94265,2.18778_mb_bins.csv,30.07,805510


In [6]:
# Latitude and longitude index to run the model
#  Longitude must be 0 - 360 degrees
latlon_all = []
for i in os.listdir(debris_prms.ostrem_fp):
    if i.endswith(debris_prms.ostrem_fn_sample.split('XXXX')[1]):
        latlon_fn = i.split(debris_prms.ostrem_fn_sample.split('XXXX')[1])[0]
        # Extract latitude
        lat_str = latlon_fn.split('-')[0]
        if 'N' in lat_str:
            i_lat = int(lat_str.split('N')[0]) / 100
        elif 'S' in lat_str:
            i_lat = -1 * int(lat_str.split('S')[0]) / 100
        # Extract longitude
        lon_str = latlon_fn.split('-')[1]
        i_lon = int(lon_str.split('E')[0]) / 100
        latlon_all.append([i_lat, i_lon, i])
latlon_all = sorted(latlon_all)

lat_all = np.array([x[0] for x in latlon_all])
lon_all = np.array([x[1] for x in latlon_all])
ostrem_fn_all_raw = [x[2] for x in latlon_all]

main_glac_rgi['lat_nearest'] = np.nan
main_glac_rgi['lon_nearest'] = np.nan
main_glac_rgi['ostrem_fn'] = np.nan
for nglac, glac_idx in enumerate(main_glac_rgi.index.values):
    latlon_dist = (((main_glac_rgi.loc[glac_idx,'CenLat'] - lat_all)**2 + 
                    (main_glac_rgi.loc[glac_idx,'CenLon_360'] - lon_all)**2)**0.5)
    latlon_nearidx = np.where(latlon_dist == latlon_dist.min())[0][0]
    
    main_glac_rgi.loc[glac_idx,'lat_nearest'] = lat_all[latlon_nearidx]
    main_glac_rgi.loc[glac_idx,'lon_nearest'] = lon_all[latlon_nearidx]
    main_glac_rgi.loc[glac_idx,'ostrem_fn'] = ostrem_fn_all_raw[latlon_nearidx]
    
ostrem_fn_all = sorted(list(np.unique(main_glac_rgi['ostrem_fn'].values)))
lat_values = np.arange(main_glac_rgi.lat_nearest.min(), main_glac_rgi.lat_nearest.max() + 0.1, 0.25)
lon_values = np.arange(main_glac_rgi.lon_nearest.min(), main_glac_rgi.lon_nearest.max() + 0.1, 0.25)

main_glac_rgi

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,glacno,rgino_str,RGIId_float,CenLon_360,bin_fn,DC_Area_%,DC_Area_v2,lat_nearest,lon_nearest,ostrem_fn
GlacNo,,,,,,,,,,,,,,,,,,,,,
0,279,RGI60-02.00280,-122.57300,49.88620,2,2,5.938,1671,2340,1949,...,280,02.00280,2.00280,237.42700,2.00280_mb_bins.csv,7.90,304435,50.50,237.50,5050N-23750E-debris_melt_curve.nc
1,736,RGI60-02.00737,-116.52800,50.21440,2,3,5.536,2237,3104,2586,...,737,02.00737,2.00737,243.47200,2.00737_mb_bins.csv,8.14,288223,50.25,243.50,5025N-24350E-debris_melt_curve.nc
2,913,RGI60-02.00914,-123.85700,50.31570,2,2,4.372,1056,2233,1486,...,914,02.00914,2.00914,236.14300,2.00914_mb_bins.csv,19.99,784568,50.25,236.25,5025N-23625E-debris_melt_curve.nc
3,1103,RGI60-02.01104,-122.60500,50.41950,2,2,3.016,1850,2539,2054,...,1104,02.01104,2.01104,237.39500,2.01104_mb_bins.csv,8.92,169332,50.50,237.50,5050N-23750E-debris_melt_curve.nc
4,1151,RGI60-02.01152,-123.89000,50.34100,2,2,12.021,1280,2505,1908,...,1152,02.01152,2.01152,236.11000,2.01152_mb_bins.csv,9.40,680881,50.25,236.00,5025N-23600E-debris_melt_curve.nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,18764,RGI60-02.18765,-121.01578,48.31119,2,4,4.845,1523,2642,2125,...,18765,02.18765,2.18765,238.98422,2.18765_mb_bins.csv,11.44,485905,48.25,239.00,4825N-23900E-debris_melt_curve.nc
172,18769,RGI60-02.18770,-121.00437,48.07234,2,4,2.125,1845,2490,2253,...,18770,02.18770,2.18770,238.99563,2.18770_mb_bins.csv,12.49,114329,48.00,239.00,4800N-23900E-debris_melt_curve.nc
173,18777,RGI60-02.18778,-121.05735,48.35698,2,4,2.924,1613,2196,1891,...,18778,02.18778,2.18778,238.94265,2.18778_mb_bins.csv,30.07,805510,48.25,239.00,4825N-23900E-debris_melt_curve.nc


In [7]:
# Process each group and derive elevation statistics for the debris cover
year_mean = np.zeros((len(lat_values), len(lon_values)))
year_std = np.zeros((len(lat_values), len(lon_values)))
year_med = np.zeros((len(lat_values), len(lon_values)))
year_mad = np.zeros((len(lat_values), len(lon_values)))
doy_mean = np.zeros((len(lat_values), len(lon_values)))
doy_std = np.zeros((len(lat_values), len(lon_values)))
doy_med = np.zeros((len(lat_values), len(lon_values)))
doy_mad = np.zeros((len(lat_values), len(lon_values)))
dayfrac_mean = np.zeros((len(lat_values), len(lon_values)))
dayfrac_std = np.zeros((len(lat_values), len(lon_values)))
dayfrac_med = np.zeros((len(lat_values), len(lon_values)))
dayfrac_mad = np.zeros((len(lat_values), len(lon_values)))


ts_fns_df = pd.read_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn)

for nlatlon, ostrem_fn in enumerate(ostrem_fn_all):
# for nlatlon, ostrem_fn in enumerate([ostrem_fn_all[96]]):
    
    main_glac_rgi_subset = main_glac_rgi[main_glac_rgi['ostrem_fn'] == ostrem_fn]
    main_glac_rgi_subset.reset_index(inplace=True, drop=True)
    
    lat_idx = np.where(main_glac_rgi_subset.loc[0,'lat_nearest'] == lat_values)[0][0]
    lon_idx = np.where(main_glac_rgi_subset.loc[0,'lon_nearest'] == lon_values)[0][0]
    
    lat_deg = lat_values[lat_idx]
    lon_deg = lon_values[lon_idx]
    
    print(nlatlon, lat_deg, lon_deg, ostrem_fn)

    doy_list = []
    year_list = []
    dayfrac_list = []
#     print('\n\nDELETE ME: SWITCH BACK!\n\n')
    for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[0]]):
        glac_str = main_glac_rgi_subset.loc[glac_idx,'rgino_str']
        rgiid = main_glac_rgi_subset.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

#         print(nglac, glac_idx, rgiid)

        # Ts filenames
        ts_fn_idx = np.where(ts_fns_df['RGIId'].values == rgiid)[0][0]
        ts_fn = ts_fns_df.loc[ts_fn_idx,'ts_fullfn']
        ts_year_fn = ts_fns_df.loc[ts_fn_idx,'ts_year_fullfn']
        ts_doy_fn = ts_fns_df.loc[ts_fn_idx,'ts_doy_fullfn']
        ts_dayfrac_fn = ts_fns_df.loc[ts_fn_idx,'ts_dayfrac_fullfn']

        
        # Process debris-covered glaciers
        if (((main_glac_rgi_subset.loc[glac_idx, 'DC_Area_%'] > debris_prms.dc_percarea_threshold) | 
             (main_glac_rgi_subset.loc[glac_idx, 'DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold))
            and '0.0' not in ts_fns_df.loc[ts_fn_idx].values):

            # Create glacier feature from ice thickness raster
            thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
            thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
            gf = create_glacfeat(thick_dir, thick_fn)

            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if os.path.exists(dc_shp_proj_fn) == False:
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()
            
            # Add layers
            gf.add_layers(dc_shp_lyr, gf_add_dhdt=False, gf_add_vel=False, 
                          gf_add_ts=True, ts_fn=ts_fn,
                          gf_add_ts_info=True, ts_year_fn=ts_year_fn, ts_doy_fn=ts_doy_fn, ts_dayfrac_fn=ts_dayfrac_fn)

            # Isolate the debris-covered areas
            gf.ts = np.ma.array(gf.ts.data, mask=np.ma.mask_or(gf.dc_mask, np.isnan(gf.ts.data)))
            gf.ts_dayfrac = np.ma.array(gf.ts_dayfrac.data, 
                                        mask=np.ma.mask_or(gf.dc_mask, np.isnan(gf.ts_dayfrac.data)))
            gf.ts_year = np.ma.array(gf.ts_year.data, mask=np.ma.mask_or(gf.dc_mask, np.isnan(gf.ts_year.data)))
            gf.ts_doy = np.ma.array(gf.ts_doy.data, mask=np.ma.mask_or(gf.dc_mask, np.isnan(gf.ts_doy.data)))

#             # ===== PLOTS =====
#             # DEM
#             var_full2plot = gf.z1.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
#             # Surface temperature
#             var_full2plot = gf.ts.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
#             # Ts Day fraction
#             var_full2plot = gf.ts_dayfrac.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-dayfrac'], 'inferno', 'ts dayfrac', close_fig=False)
#             # Ts Year
#             var_full2plot = gf.ts_year.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-year'], 'inferno', 'ts year', close_fig=False)
#             # Ts Day of year
#             var_full2plot = gf.ts_doy.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-doy'], 'inferno', 'doy', close_fig=False)
            
#         # ========================================================================================================
                    
#             print(gf.ts_year.compressed())
#             print(gf.ts_doy.compressed())
    
            doy_list.extend(list(gf.ts_doy.compressed()))
            year_list.extend(list(gf.ts_year.compressed()))
            dayfrac_list.extend(list(gf.ts_dayfrac.compressed()))

    
    if len(doy_list) > 0 and len(year_list) > 0 and len(dayfrac_list) > 0:
        # Year
        year_mean_latlon = np.mean(year_list)
        year_std_latlon = np.std(year_list)
        year_med_latlon = malib.fast_median(year_list)
        year_mad_latlon = malib.mad(year_list)
        # Day of year
        #  adjust day of year to avoid issues with comparing JD 1 (Jan 1) and JD 365 (Dec 31),
        #  since the mean would be JD 182 (June), but should be JD 0.5 (Jan 1); also would affect std, etc.
        doy_array = np.array(doy_list)
        doy_array[doy_array > 365] = 365
        doy_array[doy_array > 182] = doy_array[doy_array > 182] - 365
        doy_mean_latlon = np.mean(doy_array)
        if doy_mean_latlon < 0:
            doy_mean_latlon += 365
        doy_std_latlon = np.std(doy_array)
        doy_med_latlon = malib.fast_median(doy_array)
        if doy_med_latlon < 0:
            doy_med_latlon += 365
        doy_mad_latlon = malib.mad(doy_array)
        dayfrac_mean_latlon = np.mean(dayfrac_list)
        dayfrac_std_latlon = np.std(dayfrac_list)
        dayfrac_med_latlon = malib.fast_median(dayfrac_list)
        dayfrac_mad_latlon = malib.mad(dayfrac_list)

        # Update array
        year_mean[lat_idx,lon_idx] = year_mean_latlon
        year_std[lat_idx,lon_idx] = year_std_latlon
        year_med[lat_idx,lon_idx] = year_med_latlon
        year_mad[lat_idx,lon_idx] = year_mad_latlon
        doy_mean[lat_idx,lon_idx] = doy_mean_latlon
        doy_std[lat_idx,lon_idx] = doy_std_latlon
        doy_med[lat_idx,lon_idx] = doy_med_latlon
        doy_mad[lat_idx,lon_idx] = doy_mad_latlon
        dayfrac_mean[lat_idx,lon_idx] = dayfrac_mean_latlon
        dayfrac_std[lat_idx,lon_idx] = dayfrac_std_latlon
        dayfrac_med[lat_idx,lon_idx] = dayfrac_med_latlon
        dayfrac_mad[lat_idx,lon_idx] = dayfrac_mad_latlon

        print('  year mean +/- std:', np.round(year_mean_latlon,1), np.round(year_std_latlon,1)) 
        print('  doy mean +/- std:', np.round(doy_mean_latlon,1), np.round(doy_std_latlon,1)) 
        print('    doy median +/- mad:', np.round(doy_med_latlon,1), np.round(doy_mad_latlon,1)) 
        print('  dayfrac mean +/- std:', np.round(dayfrac_mean_latlon,3), np.round(dayfrac_std_latlon,3))

0 43.25 250.25 4325N-25025E-debris_melt_curve.nc
  year mean +/- std: 2014.9 1.3
  doy mean +/- std: 207.1 6.3
    doy median +/- mad: 209.0 2.0
  dayfrac mean +/- std: 18.032 0.014
1 43.25 250.5 4325N-25050E-debris_melt_curve.nc
  year mean +/- std: 2015.5 0.9
  doy mean +/- std: 210.6 9.0
    doy median +/- mad: 209.2 1.1
  dayfrac mean +/- std: 18.025 0.01
2 45.25 238.25 4525N-23825E-debris_melt_curve.nc
  year mean +/- std: 2015.0 0.0
  doy mean +/- std: 193.3 50.3
    doy median +/- mad: 182.8 0.0
  dayfrac mean +/- std: 18.824 0.003
3 46.25 238.5 4625N-23850E-debris_melt_curve.nc
  year mean +/- std: 2014.8 0.5
  doy mean +/- std: 240.3 88.9
    doy median +/- mad: 228.4 15.4
  dayfrac mean +/- std: 18.839 0.039
4 46.75 238.25 4675N-23825E-debris_melt_curve.nc
  year mean +/- std: 2014.8 0.6
  doy mean +/- std: 229.3 102.3
    doy median +/- mad: 186.4 5.8
  dayfrac mean +/- std: 18.846 0.051
5 47.0 238.25 4700N-23825E-debris_melt_curve.nc
  year mean +/- std: 2014.7 0.7
  doy me

  year mean +/- std: 2015.0 0.7
  doy mean +/- std: 215.0 49.0
    doy median +/- mad: 214.1 32.7
  dayfrac mean +/- std: 19.316 0.005
47 51.75 234.0 5175N-23400E-debris_melt_curve.nc
  year mean +/- std: 2015.0 1.0
  doy mean +/- std: 222.1 62.6
    doy median +/- mad: 214.5 22.0
  dayfrac mean +/- std: 19.268 0.042
48 51.75 234.25 5175N-23425E-debris_melt_curve.nc
  year mean +/- std: 2015.2 0.9
  doy mean +/- std: 222.6 10.0
    doy median +/- mad: 225.0 9.7
  dayfrac mean +/- std: 19.267 0.044
49 51.75 241.75 5175N-24175E-debris_melt_curve.nc
  year mean +/- std: 2014.5 0.5
  doy mean +/- std: 216.8 8.1
    doy median +/- mad: 213.9 8.5
  dayfrac mean +/- std: 18.756 0.051
50 51.75 243.0 5175N-24300E-debris_melt_curve.nc
  year mean +/- std: 2015.5 0.6
  doy mean +/- std: 212.3 9.0
    doy median +/- mad: 210.2 2.3
  dayfrac mean +/- std: 18.7 0.003
51 52.0 233.0 5200N-23300E-debris_melt_curve.nc
  year mean +/- std: 2015.0 0.0
  doy mean +/- std: 185.8 0.0
    doy median +/- mad: 

In [8]:
# Export to dataset
ds_ts_stats = xr.Dataset({'year_mean': (['latitude', 'longitude'], year_mean),
                          'year_std': (['latitude', 'longitude'], year_std),
                          'year_med': (['latitude', 'longitude'], year_med),
                          'year_mad': (['latitude', 'longitude'], year_mad),
                          'doy_mean': (['latitude', 'longitude'], doy_mean),
                          'doy_std': (['latitude', 'longitude'], doy_std),
                          'doy_med': (['latitude', 'longitude'], doy_med),
                          'doy_mad': (['latitude', 'longitude'], doy_mad),
                          'dayfrac_mean': (['latitude', 'longitude'], dayfrac_mean),
                          'dayfrac_std': (['latitude', 'longitude'], dayfrac_std),
                          'dayfrac_med': (['latitude', 'longitude'], dayfrac_med),
                          'dayfrac_mad': (['latitude', 'longitude'], dayfrac_mad),},
                          coords={'latitude': lat_values,
                                  'longitude': lon_values})

attrs_dict={
     'year_mean':{'units':'-',
         'long_name':'mean year',
         'comment': 'mean year when mosaicked surface temperature satellite image was acquired'},
     'year_std':{'units':'-',
         'long_name':'year standard deviation',
         'comment': 'standard deviation of year when mosaicked surface temperature satellite image was acquired'},
     'year_med':{'units':'-',
         'long_name':'median year',
         'comment': 'median year when mosaicked surface temperature satellite image was acquired'},
     'year_mad':{'units':'-',
         'long_name':'median absolute deviation year',
         'comment': 'median absolute deviation of year of when mosaicked surface temperature satellite image was acquired'},
     'doy_mean':{'units':'days since January 1',
         'long_name':'mean day of year',
         'comment': 'mean day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_std':{'units':'days since January 1',
         'long_name':'day of year standard deviation',
         'comment': 'standard deviation of day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_med':{'units':'days since January 1',
         'long_name':'median day of year',
         'comment': 'median day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_mad':{'units':'days since January 1',
         'long_name':'median absolute deviation day of year',
         'comment': 'day of year of year of when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_mean':{'units':'-',
         'long_name':'mean hour',
         'comment': 'mean hour of when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_std':{'units':'-',
         'long_name':'year standard deviation',
         'comment': 'standard deviation of hour when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_med':{'units':'-',
         'long_name':'median hour',
         'comment': 'median hour when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_mad':{'units':'-',
         'long_name':'median absolute deviation hour',
         'comment': 'median absolute deviation of hour of when mosaicked surface temperature satellite image was acquired'},}

for vn in ['year_mean', 'year_std', 'year_med', 'year_mad',
           'doy_mean', 'doy_std', 'doy_med', 'doy_mad',
           'dayfrac_mean', 'dayfrac_std', 'dayfrac_med', 'dayfrac_mad',]:
    ds_ts_stats[vn].attrs = attrs_dict[vn]
    
ds_ts_stats.to_netcdf(ts_info_fullfn)
                
print(ds_ts_stats)

<xarray.Dataset>
Dimensions:       (latitude: 62, longitude: 81)
Coordinates:
  * latitude      (latitude) float64 43.25 43.5 43.75 44.0 ... 58.0 58.25 58.5
  * longitude     (longitude) float64 230.5 230.8 231.0 ... 250.0 250.2 250.5
Data variables:
    year_mean     (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_std      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_med      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_mad      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_mean      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_std       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_med       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_mad       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    dayfrac_mean  (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    dayfrac_std   (latitude, 

In [9]:
# lat_idx = 4
# lon_idx = 2
# # lat_idx = 37
# # lon_idx = 46
# print(ds_ts_stats['year_mean'][lat_idx,lon_idx].values, ds_ts_stats['year_std'][lat_idx,lon_idx].values, 
#       ds_ts_stats['doy_mean'][lat_idx,lon_idx].values, ds_ts_stats['doy_std'][lat_idx,lon_idx].values)

In [10]:
ts_info_fullfn

'/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/../output/ts_tif/02_ts_data/02_debris_tsinfo.nc'